In [40]:
import numpy as np
import pandas as pd
from sklearn import model_selection
from sklearn import metrics
from sklearn import tree
from sklearn import linear_model 
from sklearn import ensemble 

In [2]:
data = pd.read_csv('weatherAUS.csv')

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 145460 entries, 0 to 145459
Data columns (total 23 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   Date           145460 non-null  object 
 1   Location       145460 non-null  object 
 2   MinTemp        143975 non-null  float64
 3   MaxTemp        144199 non-null  float64
 4   Rainfall       142199 non-null  float64
 5   Evaporation    82670 non-null   float64
 6   Sunshine       75625 non-null   float64
 7   WindGustDir    135134 non-null  object 
 8   WindGustSpeed  135197 non-null  float64
 9   WindDir9am     134894 non-null  object 
 10  WindDir3pm     141232 non-null  object 
 11  WindSpeed9am   143693 non-null  float64
 12  WindSpeed3pm   142398 non-null  float64
 13  Humidity9am    142806 non-null  float64
 14  Humidity3pm    140953 non-null  float64
 15  Pressure9am    130395 non-null  float64
 16  Pressure3pm    130432 non-null  float64
 17  Cloud9am       89572 non-null

In [7]:
data.isnull().sum().sum()

343248

In [10]:
round(data.isnull().sum() / len(data),3)
data.drop(['Evaporation','Sunshine','Cloud3pm'], axis = 1, inplace = True)


In [19]:
display(round(data['RainTomorrow'].mean(),2))

0.22

In [14]:
data['RainToday'] = data['RainToday'].apply(lambda x: 1 if x=='Yes' else 0)

In [16]:
data['RainTomorrow'] = data['RainTomorrow'].apply(lambda x: 1 if x=='Yes' else 0)

In [20]:
data.Date= pd.to_datetime(data.Date)
data['Month'] = data.Date.dt.month
data.drop('Date', axis = 1, inplace = True)

In [21]:
df_season = data.groupby('Month').mean()
df_season[['RainToday']]

,RainToday
Month,
1,0.184875
2,0.201612
3,0.211885
4,0.212208
5,0.217255
6,0.257569
7,0.265198
8,0.247589
9,0.223368


In [22]:
categoricals = ['Month', 'Location', 'WindGustDir', 'WindDir9am', 'WindDir3pm']
df_dummies = pd.get_dummies(data, columns=categoricals)
df_dummies.shape

(145460, 124)

In [23]:
df_dummies

,MinTemp,MaxTemp,Rainfall,WindGustSpeed,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,...,WindDir3pm_NNW,WindDir3pm_NW,WindDir3pm_S,WindDir3pm_SE,WindDir3pm_SSE,WindDir3pm_SSW,WindDir3pm_SW,WindDir3pm_W,WindDir3pm_WNW,WindDir3pm_WSW
0,13.4,22.9,0.6,44.0,20.0,24.0,71.0,22.0,1007.7,1007.1,...,0,0,0,0,0,0,0,0,1,0
1,7.4,25.1,0.0,44.0,4.0,22.0,44.0,25.0,1010.6,1007.8,...,0,0,0,0,0,0,0,0,0,1
2,12.9,25.7,0.0,46.0,19.0,26.0,38.0,30.0,1007.6,1008.7,...,0,0,0,0,0,0,0,0,0,1
3,9.2,28.0,0.0,24.0,11.0,9.0,45.0,16.0,1017.6,1012.8,...,0,0,0,0,0,0,0,0,0,0
4,17.5,32.3,1.0,41.0,7.0,20.0,82.0,33.0,1010.8,1006.0,...,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145455,2.8,23.4,0.0,31.0,13.0,11.0,51.0,24.0,1024.6,1020.3,...,0,0,0,0,0,0,0,0,0,0
145456,3.6,25.3,0.0,22.0,13.0,9.0,56.0,21.0,1023.5,1019.1,...,0,0,0,0,0,0,0,0,0,0
145457,5.4,26.9,0.0,37.0,9.0,9.0,53.0,24.0,1021.0,1016.8,...,0,0,0,0,0,0,0,0,1,0
145458,7.8,27.0,0.0,28.0,13.0,7.0,51.0,24.0,1019.4,1016.5,...,0,0,0,0,0,0,0,0,0,0


In [24]:
df_dummies.dropna(inplace=True)

In [27]:
X = df_dummies.drop('RainTomorrow', axis = 1)
Y = df_dummies['RainTomorrow']  
X_train, X_test, Y_train, Y_test = model_selection.train_test_split(X, Y, test_size=0.3, random_state = 31)
Y_test.mean()

0.22515547658332274

In [28]:
def gbs(data, n):     
    inds = np.random.randint(0, len(data), (n, len(data))) #определяем индексы случайным образом
    numbers = data[inds] #выбираем значения по индексам
    return numbers
target = X_train['MinTemp'].values #выбираем целевую переменную
np.random.seed(31) #задаём параметр генератора случайных чисел
mean_values = [np.mean(x) for x in gbs(target, 1000)] #получаем все средние значения
np.std(mean_values) #находим для них стандартное отклонение

0.027644796745379473

In [31]:
clf = linear_model.LogisticRegression()
clf.fit(X_train, Y_train)
preds_train = clf.predict(X_train)
preds_test = clf.predict(X_test)
metrics.roc_auc_score(Y_test, preds_test)

c:\Users\askar\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.7256758603432787

In [32]:
params = {'max_leaf_nodes': list(range(2, 10)), 'min_samples_split': [2, 3, 4], 'max_depth': [5,7,9,11]}

In [36]:
grid_search_cv = model_selection.GridSearchCV(tree.DecisionTreeClassifier(random_state=42), params, verbose=3, cv=3)
grid_search_cv.fit(X_train, Y_train)
print(grid_search_cv.best_params_)

Fitting 3 folds for each of 96 candidates, totalling 288 fits
[CV 1/3] END max_depth=5, max_leaf_nodes=2, min_samples_split=2;, score=0.820 total time=   0.7s
[CV 2/3] END max_depth=5, max_leaf_nodes=2, min_samples_split=2;, score=0.825 total time=   1.0s
[CV 3/3] END max_depth=5, max_leaf_nodes=2, min_samples_split=2;, score=0.818 total time=   0.7s
[CV 1/3] END max_depth=5, max_leaf_nodes=2, min_samples_split=3;, score=0.820 total time=   0.7s
[CV 2/3] END max_depth=5, max_leaf_nodes=2, min_samples_split=3;, score=0.825 total time=   0.7s
[CV 3/3] END max_depth=5, max_leaf_nodes=2, min_samples_split=3;, score=0.818 total time=   0.6s
[CV 1/3] END max_depth=5, max_leaf_nodes=2, min_samples_split=4;, score=0.820 total time=   0.5s
[CV 2/3] END max_depth=5, max_leaf_nodes=2, min_samples_split=4;, score=0.825 total time=   0.4s
[CV 3/3] END max_depth=5, max_leaf_nodes=2, min_samples_split=4;, score=0.818 total time=   0.7s
[CV 1/3] END max_depth=5, max_leaf_nodes=3, min_samples_split=2;,

In [37]:
clf = tree.DecisionTreeClassifier(max_depth = 5, max_leaf_nodes = 9, min_samples_split = 2, random_state=42)
clf.fit(X_train, Y_train)
preds_train = clf.predict(X_train)
preds_test = clf.predict(X_test)
print(round(metrics.roc_auc_score(Y_test, preds_test), 2))

0.71


In [41]:
clf = ensemble.RandomForestClassifier(n_estimators = 100, random_state=31)
clf.fit(X_train, Y_train)
preds_train = clf.predict(X_train)
preds_test = clf.predict(X_test)
print(round(metrics.roc_auc_score(Y_test, preds_test), 2))

0.74


In [42]:
params = {
    'max_features': [4, 5, 6, 7], 
    'min_samples_leaf': [3, 5, 7, 9, 11], 
    'max_depth': [5, 10, 15]
}
grid_search_cv = model_selection.GridSearchCV(ensemble.RandomForestClassifier(random_state=31), params, verbose=3, cv=3)
grid_search_cv.fit(X_train, Y_train)
print(grid_search_cv.best_params_)
clf = ensemble.RandomForestClassifier(max_depth=15, max_features=7, min_samples_leaf=3, random_state=31)
clf.fit(X_train, Y_train)
preds_train = clf.predict(X_train)
preds_test = clf.predict(X_test)
print(round(metrics.roc_auc_score(Y_test, preds_test), 2))

Fitting 3 folds for each of 60 candidates, totalling 180 fits
[CV 1/3] END max_depth=5, max_features=4, min_samples_leaf=3;, score=0.777 total time=   4.5s
[CV 2/3] END max_depth=5, max_features=4, min_samples_leaf=3;, score=0.778 total time=   4.3s
[CV 3/3] END max_depth=5, max_features=4, min_samples_leaf=3;, score=0.777 total time=   3.3s
[CV 1/3] END max_depth=5, max_features=4, min_samples_leaf=5;, score=0.777 total time=   2.2s
[CV 2/3] END max_depth=5, max_features=4, min_samples_leaf=5;, score=0.778 total time=   3.8s
[CV 3/3] END max_depth=5, max_features=4, min_samples_leaf=5;, score=0.777 total time=   3.2s
[CV 1/3] END max_depth=5, max_features=4, min_samples_leaf=7;, score=0.777 total time=   2.8s
[CV 2/3] END max_depth=5, max_features=4, min_samples_leaf=7;, score=0.777 total time=   4.3s
[CV 3/3] END max_depth=5, max_features=4, min_samples_leaf=7;, score=0.777 total time=   2.5s
[CV 1/3] END max_depth=5, max_features=4, min_samples_leaf=9;, score=0.777 total time=   2.5

In [43]:
feature_names = [x for x in df_dummies if x != 'RainTomorrow']
pd.DataFrame({'feat': feature_names,
              'coef': clf.feature_importances_}).sort_values(by='coef', ascending=False)

,feat,coef
7,Humidity3pm,0.249488
2,Rainfall,0.081319
10,Cloud9am,0.065026
9,Pressure3pm,0.064832
8,Pressure9am,0.062589
...,...,...
62,Location_SalmonGums,0.000000
43,Location_Launceston,0.000000
72,Location_Witchcliffe,0.000000
66,Location_Tuggeranong,0.000000
